<a href="https://colab.research.google.com/github/yenlung/AI-Demo/blob/master/%E3%80%90Demo04b%E3%80%91%E5%93%A1%E7%91%9B%E5%BC%8F%E6%80%9D%E8%80%83%E7%94%9F%E6%88%90%E5%99%A8_Two_Stage_CoT_%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 安裝需要的套件

In [1]:
!pip install openai gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

### 2. 設定 OpenAI API 金鑰

In [10]:
# 準備讀入存在 Colab 的金鑰

import os
import openai
from openai import OpenAI

from google.colab import userdata

In [12]:
api_key = userdata.get('OpenAI')
openai.api_key = api_key

In [7]:
model = "gpt-4o"
# "gpt-3.5-turbo"

### 3. 定義第一次 CoT 思考函數

In [19]:
def generate_thoughts(event_description):
    prompt = f"""
    你是一個充滿創意與正能量的助手。
    使用者遇到這個事件：「{event_description}」

    請你用輕鬆、有趣的語氣，想出這件事是「超級幸運」且「有趣」的 5 個理由。
    以條列式（1. 2. 3. 4. 5.）列出你的想法，請用第一人稱，也就是幫使用者想理由。
    """

    response = openai.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.9,
        #max_tokens=300
    )

    thoughts = response.choices[0].message.content
    return thoughts

我們可以測試這個函數。

In [24]:
prompt = "今天 Uber 送錯餐，把別人的餐送給了我。"
thoughts = generate_thoughts(prompt)

In [28]:
print(thoughts)

1. 嘿嘿，今天的我居然得到了免費的「神秘禮物餐」！就像人生的一場美味幸運抽獎，期待每天都有驚喜上門！

2. 終於有機會一嘗別人喜歡的菜色，說不定發現了新寵愛的美食。這可是平時不敢輕易嘗試的料理，豐富了我的味蕾冒險！

3. 這次的「餐點差錯」讓我有絕佳的故事可以分享，在聚會或約會時總能逗樂朋友，成為話題中心！

4. 我的生活瞬間成了一部小情境喜劇，就像在電視劇裡看到的搞笑情節，讓我的一天增添了幾分戲劇性。

5. 這次的送錯餐讓我特別感激自己原本的訂單，下次會更珍惜每一口熟悉的美味，果然「失而復得」的感覺就是這麼奇妙！


### 4. 定義第二次正式回應函數

In [48]:
def generate_final_reply(prompt, thoughts):
    prompt_two = f"""
    我遇到了這個事件: 「{prompt}」, 這件事有 5 個理由, 其實是超幸運的事。
    {thoughts}

    請從這 5 個理由中，選出最有趣且最能令人感到愉快的一個，然後用更生動活潑的語氣, 以我的口吻、社群媒體 po 文的口吻說一次, 說為什麼這是一件超幸運的事, 並且以「完全是 Lucky Vicky 呀!」結尾。
    """

    response = openai.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt_two}],
        temperature=0.9,
        max_tokens=200
    )

    final_reply = response.choices[0].message.content
    return final_reply

In [49]:
final_reply = generate_final_reply(prompt, thoughts)

In [50]:
print(final_reply)

哈哈，今天我超幸運地收到了免費的「神秘禮物餐」！簡直就像中了美食界的樂透，人生美味的幸運抽獎就在我眼前展開！想想看，每天都有驚喜美食上門，這種感覺就像拆開一個未知的禮物包裹，讓人心情大好！這次的事件讓我真切感受到，生活中的小意外竟然能帶來如此大的快樂和意想不到的驚喜！完全是 Lucky Vicky 呀！🎉🥳🍽️✨


### 5. 用 Gradio 打造 Web App!

In [53]:
import gradio as gr

In [51]:
def two_stage_cot(prompt):
    thoughts = generate_thoughts(prompt)
    final_reply = generate_final_reply(prompt, thoughts)
    return thoughts, final_reply

In [56]:
iface = gr.Interface(
    fn=two_stage_cot,
    inputs=gr.Textbox(label="輸入", placeholder="例如：今天走路撿到了一片葉子"),
    outputs=[
        gr.Textbox(label="🤔 思考"),
        gr.Textbox(label="✨ 正式回應")
    ],
    title="員瑛式思考生成器",
    description="嗨！我是你的「員瑛式思考生成器」，專門讓任何事情都能閃閃發亮✨～有什麼事最近讓你覺得有點不順，或是生活中的小小插曲？沒關係，不管大事小事，我都會幫你找到其中的好運氣！🌈無論是忘記帶鑰匙、沒趕上公車，或是工作上遇到困難，其實都可能是幸運的轉折點呢！  只要和我分享一下，我就會用「員瑛式思考」讓這些事變得正能量滿滿，並告訴你為什麼這一切都是「完全是 Lucky Vicky 呀！」"
)

In [57]:
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://77dac287ee6401823f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
